In [22]:
import datetime
import pandas as pd
import numpy as np
import random as rand
from bayes_opt import BayesianOptimization #pip install bayesian-optimization

#테스트 파일 로딩
coin_name = "KRW-XRP"
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
print(df_org.shape)
print(df_org.columns)

(199878, 31)
Index(['t', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99', 'wma7', 'wma25',
       'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s', 'macd_d', 'rsi7',
       'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh', 'bhi', 'bl', 'bli',
       'bm', 'bw', 'vwap'],
      dtype='object')


In [23]:
#타임스켐프 시간 변환
def get_time_hhmmss(mili_time):
    mili_time = float(mili_time)/1000
    KST = datetime.timezone(datetime.timedelta(hours=9))
    dt = datetime.datetime.fromtimestamp(mili_time, tz=KST)
    timeline = str(dt.strftime('%D %H:%M:%S'))
    return timeline

#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)

#수익률 보정: 총매수금액 200 coin(xrp) 기준으로 조정
def adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate):
    open_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_amt = open_amt_list[len(open_amt_list)-1]
    adj_revenue = (200 * revenue) / max_amt
    return adj_revenue

def run_test(config):
    revenue_rate  = config['revenue_rate']
    max_loss_rate = config['max_loss_rate']
    increace_rate = config['increace_rate']
    buy_cnt_limit = int(config['buy_cnt_limit'])
    buy_amt_unit  = config['buy_amt_unit']
    trade_fee = 0.001 #거래수수료
    close = 1300
    buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)

    buy_cnt = 0 
    buy_price = 0
    buy_amt = 0
    revenue = 0
    revenue_t = 0

#     df = df_org.iloc[df_org.shape[0]-28800:]
    df = df_org.iloc[df_org.shape[0]-144000:df_org.shape[0]-124000,]
    # df = df_org2
    for i in range(0, df.shape[0]-1):
        close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
        close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4) #다음 분의 종가로 산다
        wma7 = round(df.iloc[i:i+1,]['wma7'].values[0],4)
        wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4)
        vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
        macd_s = round(df.iloc[i:i+1,]['macd_s'].values[0],4)
        macd_d = round(df.iloc[i:i+1,]['macd_d'].values[0],4)

        #stop loss
        loss = buy_price - close2*buy_amt
        if loss > max_loss:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #take profit
        tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
        if buy_cnt > 0 and tp_revenue > 0:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #buy
        if buy_cnt < buy_cnt_limit and close2 < vwap and close2 < wma7 and wma7 > wma99:
            temp_amt = buy_amt_unit + buy_amt*increace_rate
            buy_price = round(buy_price + close2*temp_amt,4)
            buy_amt = round(buy_amt + temp_amt,4)
            buy_cnt = buy_cnt + 1 
        
    return adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate)

In [24]:
def black_box_function(revenue_rate, max_loss_rate, increace_rate, buy_cnt_limit, buy_amt_unit):
    config_data = {
        'revenue_rate':revenue_rate,
        'max_loss_rate':max_loss_rate,
        'increace_rate':increace_rate,
        'buy_cnt_limit':buy_cnt_limit,
        'buy_amt_unit':buy_amt_unit
    }
    revenue = run_test(config_data)
    return revenue

pbounds = {
    'revenue_rate':(0.005,0.015),
    'max_loss_rate':(0.05,0.3),
    'increace_rate':(0.1,0.4),
    'buy_cnt_limit':(5,20),
    'buy_amt_unit':(5,40)
    
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1
)

#실행
optimizer.maximize(
    init_points=5,
    n_iter=50
)

|   iter    |  target   | buy_am... | buy_cn... | increa... | max_lo... | revenu... |
-------------------------------------------------------------------------------------
|  1        | -1.043e+0 |  19.6     |  15.8     |  0.1      |  0.1256   |  0.006468 |
|  2        | -1.566e+0 |  8.232    |  7.794    |  0.2037   |  0.1492   |  0.01039  |
|  3        | -9.999e+0 |  19.67    |  15.28    |  0.1613   |  0.2695   |  0.005274 |
|  4        | -5.43e+04 |  28.47    |  11.26    |  0.2676   |  0.0851   |  0.006981 |
|  5        | -1.527e+0 |  33.03    |  19.52    |  0.194    |  0.2231   |  0.01376  |
|  6        | -1.167e+0 |  19.42    |  15.8     |  0.1481   |  0.0956   |  0.0143   |
|  7        | -1.573e+0 |  15.5     |  11.88    |  0.2758   |  0.2358   |  0.009742 |
|  8        | -1.122e+0 |  30.77    |  15.36    |  0.1288   |  0.2187   |  0.01152  |
|  9        | -4.456e+0 |  18.6     |  18.89    |  0.3546   |  0.1966   |  0.007194 |
|  10       | -1.402e+0 |  7.831    |  17.75    |  0.3

In [25]:
target_list = []
i=0
for res in optimizer.res:
    target_list.append([res["target"], i])
    i=i+1
target_list.sort(reverse=True)    
target_list

[[35089.52022697153, 39],
 [-3028.1005287986013, 52],
 [-10429.223830076946, 0],
 [-16755.898463057336, 49],
 [-35727.25586903055, 32],
 [-44562.69460813377, 8],
 [-53377.91308846494, 47],
 [-54011.75452575025, 15],
 [-54304.06882509695, 3],
 [-54528.53013128329, 31],
 [-55021.357518112374, 53],
 [-69486.8542810619, 22],
 [-89999.37028636695, 46],
 [-93451.41427704354, 16],
 [-93864.51167237648, 23],
 [-95967.06770628932, 24],
 [-99991.56004146792, 2],
 [-100189.28183003112, 12],
 [-101585.01116354612, 10],
 [-102139.18046466846, 43],
 [-104231.42905633629, 18],
 [-109194.46568308744, 28],
 [-112246.0391250576, 7],
 [-116725.56262223661, 5],
 [-119927.08770118107, 35],
 [-129535.26618313298, 38],
 [-129898.20624233632, 40],
 [-130247.78129415232, 41],
 [-130717.09751535743, 45],
 [-130998.96116113503, 50],
 [-131718.58109795782, 19],
 [-133333.38679902372, 14],
 [-134340.75160325126, 25],
 [-140179.73748277317, 9],
 [-148758.4840133946, 36],
 [-148811.33683293077, 27],
 [-151122.399432

In [26]:
optimizer.res[39]

{'target': 35089.52022697153,
 'params': {'buy_amt_unit': 6.247376911501856,
  'buy_cnt_limit': 19.158806473699883,
  'increace_rate': 0.3210436156302241,
  'max_loss_rate': 0.09070804720590168,
  'revenue_rate': 0.005967624387149014}}